# Heapsort

The heapsort algorithm solves the same problem as [insertion sort](insertion_sort.ipynb).

Heapsort algorithm sorts $n$ arbitrary real numbers **in place** in $O(n log n)$ time. 

It uses an important data structure: **heap**, with which one can also implement a priority queue.

## Comparison sort

Insertion sort, merge sort, heapsort, and quicksort are all comparison sorts: they determine the sorted order of an input array by comparing elememts. 

## Heaps

The **(binary) heap** data structure is an array object that we can view as a **nearly** complete binary tree. Each node of the tree corresponds to an element of the array. The tree is completely filled on all levels except possibly the lowest, which is filled from the left up to a point. 

An array `A` that represents a heap is an object with the 2 attributes:

1. `A.length`, the number of elements in the array
2. `A.heap_size` the number of elements in the heap

The root of the heap is `A[1]` and given the index $i$ of a node, we can easily compute the indices of its parent, left child, and right child:
```python
def parent(i):
    return i >> 1

def left(i) 
    return i << 1

def right(i) 
    return (i << 1) + 1

```
## Max-heap, min-heap

The property for max-heap is that for every node `i` other than the root, `A[parent(i)] ≥ A[i]`. That is, the value of a node is at most the value of its parent. Thus, the largest element in the heap is stored at the root. 

The property for a min-heap is the other way around: `A[parent(i)] ≤ A[i]`. 

The **height** of a node is the longest downward path from it to a leaf, and the **height** of a heap is the height of its root. 

In heapsort, we typically use a max-heap, and min-heaps are commonly used to implement priority queues. 

## Maintain the heap property

In order to maintain the max-heap property, we need a procedure `heapify()`. Its inputs are an array `A` and an index `i` into the array. When it is called, `heapify()` assumes that the binary trees rooted at `left(i)` and `right(i)` are max-heaps, but that `A[i]` might be smaller than its children, thus violating the max-heap property. 

`heapify()` then "sinks" `A[i]` in the max-heap so that the subtree rooted at index `i` obeys the property.

```python
def heapify(arr, i):
    l = left(i)
    r = right(i)
    if l <= arr.heap_size and arr[l] > arr[i]:
        largest = l
    else:
        largest = i
    if r <= arr.heap_size and arr[r] > arr[i]:
        largest = r
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, largest)
```
The above psudocode runs in $O(lg n)$ time:

- fixing up the relationships among `A[i], A[left], A[right]` takes up $O(1)$ time;

- the subtree processed by the recursive subroutine should have at most the size of $2n \over 3$, and this worst case occurs when the bottom level of the tree is exactly half full. 

Thus we describe the running time by the recurrence:
$T(n) ≤ T({2n \over 3}) + O(1)$

Applying the [Master Method](big_o_notation.ipynb) we have $T(n) = O(log n)$

## Build a heap

Consider an array `A` that is already a heap, it's easy to know that all elements in the subarray `A[n/2 + 1 : n]` are all leaves of the tree, and so each is a 1-element heap to begin with. The procedure `build_heap()` goes through the remaining nodes of the tree and runs `heapify()` on each node.

```python
def build_heap(arr):
    arr.heap_size = arr.length
    for i in range(arr.length / 2, -1, -1):
        heapify(arr, i)
```
A sparse estimation of the time upper bound is $O(n log n)$, we can derive a tighter bound by observing that the time for `heapify()` to run at a node varies with the height of the node in the tree, and the heights of most nodes are small. Our tighter analysis relies on the properties that an n-element heap has height $log n$ and at most $n / 2^{h+1}$ nodes of any height $h$. 

$$
\sum_{h=0}^{log n} {n \over {2 ^ {h + 1}}} O(h) = O(n \sum_{h=0}^{log n} {h \over 2^{h}})
$$
Evaluating the last summation as follows:

Let $k=logn, S=\sum_{h=0}^{k} {h \over {2^{h+1}}}$, So that $2S=\sum_{h=0}^{k} {h \over {2^{h}}}$.

$2S - S=({1\over2}+{2\over4}+{3\over8}+...+{k\over{2^k}}) - ({1\over4}+{2\over8}+...+{{k-1}\over{2^k}} + {k\over{2^{k+1}}})$

$S={1\over2} + {1\over4} + {1\over8} + ... + {1\over{2^k}} - {k\over{2^{k+1}}}$

It's easy to know that 
$$S \lt 1 - {k\over{2^{k+1}}} \lt 1$$

Therefore $O\Big( n \sum_{h=0}^{logn} {h \over {2 ^ {h + 1}}} \Big)=O(n \times S)=O(n)$


## The heapsort in flesh!

Heapsort has the following steps:

1. build a max-heap on the input array using `build_heap()`
2. exchange `A[n]` with `A[1]` and decrement `heap_size` by 1
3. execute `heapify(A, 1)` since the exchanged `A[1]` might violate the max-heap property
4. repeat step 2 and 3 until `heap_size` is downto 2

Without further ado, we implement the algorithm with the actual code:

In [1]:
def heapsort(arr):
    def heapify(arr, i, heap_size):
        l, r = i << 1, (i << 1) + 1
        if l <= heap_size and arr[l] > arr[i]:
            largest = l
        else:
            largest = i
        if r <= heap_size and arr[r] > arr[largest]:
            largest = r
        if largest == i:
            return
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, largest, heap_size)
    
    def build_heap(arr):
        heap_size = len(arr) - 1
        for i in range(len(arr) // 2, 0, -1):
            heapify(arr, i, heap_size)
    
    arr.insert(0, 0)
    build_heap(arr)
    heap_size = len(arr) - 1
    for i in range(len(arr) - 1, 1, -1):
        arr[1], arr[i] = arr[i], arr[1]
        heap_size -= 1
        heapify(arr, 1, heap_size)
    arr.pop(0)

    
a = [16, 1, 10, 8, 7, 9, 3, 2, 4, 14]
heapsort(a)
print(a)

[1, 2, 3, 4, 7, 8, 9, 10, 14, 16]
